# 2023-12-10

In [ ]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2023, day=10)


## Initial solution
Did not do it on time today. Part 1 felt easy, just BFS and stopping when you reach a peak (neighbors should have same dist value). Part 2 was really difficult. I didn't know the theory required, such as:

[Even–odd rule](https://en.wikipedia.org/wiki/Even%E2%80%93odd_rule) <br>
[Jordan curve theorem](https://en.wikipedia.org/wiki/Jordan_curve_theorem)

I had to look these up. I spent a really long time on day 2 today. Implementation was difficult due to the discret nature and edge cases.
```
      --------Part 1--------   --------Part 2--------
Day       Time   Rank  Score       Time   Rank  Score
 10   08:57:49  24480      0   19:37:54  25658      0

```

### Puzzle 1

In [ ]:
import numpy as np

data = puzzle.input_data
s_index = data.replace("\n", "").find("S")
n_cols, n_rows = len(data.splitlines()[0]), len(data.splitlines())
s_row, s_col = s_index//n_cols, s_index % n_cols

grid = np.array([list(x) for x in data.splitlines()])

table = {
    "|" : lambda r, c: [(r+1, c), (r-1, c)],
    "-" : lambda r, c: [(r, c-1), (r, c+1)],
    "L" : lambda r, c: [(r, c+1), (r-1, c)],
    "J" : lambda r, c: [(r-1, c), (r, c-1)],
    "7": lambda r, c: [(r, c-1), (r+1, c)],
    "F": lambda r, c: [(r+1, c), (r, c+1)],
    ".": lambda r, c: [],
    "S": lambda r, c: [(r, c+1), (r, c-1), (r+1, c), (r-1, c)],
}

def in_bounds(r, c):
    return 0 <= r < n_rows and 0 <= c < n_cols

def find_loop_max(grid):
    queue = []
    visited = set()
    dist_matrix = np.zeros_like(grid, dtype=int)
    # Do the first step manually
    potential_start_neighbours = [x for x in table.get(grid[s_row, s_col])(s_row, s_col) if in_bounds(*x)]
    queue += [x for x in potential_start_neighbours if (s_row, s_col) in table.get(grid[x])(*x)]
    visited |= {(s_row, s_col)}
    while queue:
        r, c = queue.pop(0)
        visited |= {(r, c)}
        neighbours = [x for x in table.get(grid[r, c])(r, c) if in_bounds(*x)]
        if not neighbours:
            continue
        dist_matrix[r, c] = min([dist_matrix[x] for x in neighbours if x in visited]) + 1
        queue += [x for x in neighbours if x not in visited]
        if dist_matrix[neighbours[0]] != 0 and len(neighbours) == 2 and dist_matrix[neighbours[0]] == dist_matrix[neighbours[1]]:
            return (r, c), dist_matrix[r, c], dist_matrix, visited
    raise Exception("No loop found")

point, max_val, dist_matrix, visited = find_loop_max(grid)
puzzle.answer_a = int(max_val)


### Puzzle 2

In [ ]:
inside = set()

for r in range(n_rows):
    for c in range(n_cols):
        if (r, c) in visited:
            continue
        crossings = 0
        verticies = []
        for c_ray in range(c, n_cols):
            if (r, c_ray) in visited and grid[r, c_ray] == "|":
                crossings += 1
            if (r, c_ray) in visited and grid[r, c_ray] in ["S", "L", "J", "7", "F"]:
                verticies.append(grid[(r, c_ray)])
            if len(verticies) == 2:
                if "".join(verticies) in ["FJ", "L7", "S7", "SJ", "FS", "LS"]:
                    crossings += 1
                verticies = []
        if crossings % 2 == 1:
            inside |= {(r, c)} 
puzzle.answer_b = len(inside)
